In [2]:
import datetime
import dateutil
import tda
import atexit
import pandas as pd
import plotly.express as plotly

In [16]:
account_id='' #enter your accountID from the TDA website
transactions_data = pd.read_pickle('transactions_data_'+account_id)

In [17]:
# This assumes each position is closed
trades=transactions_data.sort_values(['transactionItem.instrument.symbol','transactionDate']).groupby(['transactionItem.instrument.symbol'])

In [18]:
trademetrics=pd.DataFrame()
trademetrics['ProfitLoss_net']=trades['netAmount'].sum()
trademetrics['ProfitLoss']=trades['transactionItem.cost'].sum()
trademetrics['ProfitLoss_pct']=trades['transactionItem.cost'].sum()/trades['transactionItem.cost'].nth(0)*100
trademetrics['daysHeld']=trades['transactionItem.instrument.daysToExpiration'].max()-trades['transactionItem.instrument.daysToExpiration'].min()
trademetrics['StartDate']=trades['transactionDate'].min()
trademetrics['EndDate']=trades['transactionDate'].max()

In [3]:
fig=plotly.scatter(trademetrics, x="daysHeld",y="ProfitLoss_pct")
fig.show()

NameError: name 'trademetrics' is not defined

In [21]:
fig=plotly.scatter(trademetrics, x="EndDate",y="ProfitLoss_net")
fig.show()

In [22]:
trademetrics=trademetrics.sort_values('EndDate')
trademetrics['RunningPL']=trademetrics['ProfitLoss_net'].cumsum()
fig=plotly.scatter(trademetrics, x="EndDate",y="RunningPL")
fig.data[0].update(mode='markers+lines')
fig.show()

In [23]:
trademetrics.loc[trademetrics['ProfitLoss_net']>0,'WinLoss']='Win'
trademetrics.loc[trademetrics['ProfitLoss_net']<0,'WinLoss']='Loss'

In [24]:
win_pct=trademetrics[trademetrics['WinLoss']=='Win']['WinLoss'].count()/trademetrics['WinLoss'].count()
loss_pct=trademetrics[trademetrics['WinLoss']=='Loss']['WinLoss'].count()/trademetrics['WinLoss'].count()

In [25]:
fig = plotly.histogram(trademetrics['WinLoss'],x="WinLoss")
fig.show()